In [7]:
import random
import matplotlib.pyplot as plt
import numpy as np

# <center> TP1 - Mariage stable </center>
<center> 2023/2024 - T. Godin, L. Naert </center>
<center> IUT de Vannes, BUT Informatique </center>

## Généralités (Séance 1)
Le problème des mariages stables consiste à trouver une façon stable de mettre en couple les éléments d'une population $P_1$ avec les éléments d'une autre population $P_2$ sachant que chaque élément de $P_1$ et de $P_2$ fournit une liste de ses préférences pour le couplage. 

L'exemple classique est l'affectation d'étudiants dans une formation après le bac (problème de ParcoursSup). Les étudiants font un classement des formations post-bac, les formations font un classement des étudiants et l'on cherche des mariages stables. 

Une situation est dite instable s'il y a au moins un étudiant et une formation post-bac qui préféreraient se mettre en couple plutôt que de rester avec leurs "partenaires" actuels. 

__Par exemple__ : `Jean-Pierre` est affecté à l'IUT de _Lannion_ et `Robert` à l'IUT de _Vannes_ alors que `Jean-Pierre` préfère l'IUT de _Vannes_ à l'IUT de _Lannion_ et l'IUT de _Vannes_ préfère `Jean-Pierre` à `Robert`. 

Durant ce TP, vous implémenterez l'algorithme de Gale–Shapley qui permet de trouver une solution stable au problème des mariages. Vous appliquerez cet algorithme au problème des affectations d'étudiants dans des formations. 

Le code ci-dessous donne des exemples de listes de préférence : 

In [8]:
etudiantPref=[[1,0,2,4,3],[0,2,1,4,3],[2,0,4,1,3],[2,1,4,3,0],[0,1,2,3,4]]

formationPref=[[2,0,4,3,1],[1,0,4,3,2],[4,1,2,3,0],[1,0,4,3,2],[2,1,4,3,0]]

<tt>etudiantPref</tt> liste les préférences de chaque étudiant.

<tt>formationPref</tt> liste les préférences de chaque formation.

Il y a donc 5 étudiants (<tt>len(etudiantPref)</tt>) et 5 formations (<tt>len(formationPref)</tt>)

L'étudiant n°0 préfère la formation n°1, puis la n°0, la n°2, la n°4 et enfin, la n°3. 

L'étudiant n°1 préfère la formation n°0, puis la n°2, la n°1, la n°4 et enfin, la n°3. 
etc.

Idem pour les formations.


__Pour chacune des questions avec un <tt>try/assert</tt>, nous vous demandons de rajouter des cas de test pertinents.__

> __Question 1 (mise en place)__ : 
Ecrire un bout de code pour afficher la liste des préférences de l'étudiant n°2, la liste des préférences de la formation n°4 et la formation préféré de l'étudiant n°3. 

> __Question 2 (préférence)__ : Ecrire une fonction `prefer(pref,c1,c2)` qui renvoie `True` si `c1` est préféré à `c2` d'après la liste des préférence `pref`.

In [9]:
def prefer(pref,c1,c2):
    """
    renvoie True si c1 est préféré à c2 d'après la liste des préférence pref.


   :param array pref: la liste de préférences
   :param int c1: premier individu
   :param int c1: deuxième individu   
   :return: True si c1 est préféré à c2
   :rtype: bool
    """
    return False
        
try:
    assert prefer(etudiantPref[0],0,1) == False
    assert prefer(etudiantPref[1],0,1) == True
    assert prefer(etudiantPref[2],0,1) == True
    assert prefer(etudiantPref[3],0,1) == False
    assert prefer(etudiantPref[4],0,1) == True
    print("prefer : OK")
except:
    print("prefer : ERREUR")

prefer : ERREUR


Un mariage est modélisé par une liste de __formations__. 
Par exemple, avec la liste des étudiants et la liste des formations définies précédemment, un mariage (partiel) pourrait être : `[1,-1,0,4,3]`. Il désigne les couples (étudiant,formation) suivants : (0,1), (2,0), (3,4), (4,3). L'étudiant 1 et la formation 2 sont encore célibataires.

> __Question 3 (mariage)__ : Ecrire une fonction `marriedTo(marriage, e, isStudent)` qui renvoie l'identifiant du partenaire de <tt>e</tt> d'après la liste de mariages <tt>marriage</tt>, et -1 s'il n'est pas marié. Attention : <tt>e</tt> est un étudiant si <tt>isStudent</tt> est à <tt>True</tt> et une formation sinon. 

In [10]:
def marriedTo(marriage,e,isStudent):
    """
    renvoie l'identifiant du partenaire de e d'après la liste des mariages marriage,
    et -1 s'il n'est pas marié

   :param array mariage: la liste des mariages actuels
   :param int e: étudiant ou formation
   :param bool isStudent: permet de connaitre le rôle de e 
   :return: l'identifiant du partenaire de e d'après la liste des mariages marriage
   :rtype: int
    """
    return -1

        
try:
    mariage = [1,-1,0,4,3]  
    assert marriedTo(mariage,0,True) == 1 #L'étudiant 0 est marié à  1
    assert marriedTo(mariage,1,True) == -1 #L'étudiant 1 n'est pas marié
    assert marriedTo(mariage,0,False) == 2 #La formation 0 est mariée à  2
    assert marriedTo(mariage,2,False) == -1 #La formation 0 n'est pas mariée
    print("marriedTo : OK")
except:
    print("marriedTo : ERREUR")

marriedTo : ERREUR


## Algorithme de Gale-Shapley

L'algorithme de Gale-Shapley permet de donner une solution au problème des mariages stables. Il fonctionne de manière itérative jusqu'à ce que chacun ait un partenaire.

Au départ, les étudiants et les formations ne sont pas appariés.

Pour chaque étudiant non "marié" : 

- l'étudiant propose le "mariage" à la formation en haut de sa liste (indice = 0) : 

    - Si la formation est "célibataire", elle accepte le mariage.
    - Si la formation est "mariée" : 
        - Si le couple précédent est instable, elle accepte le nouveau mariage
        - sinon l'étudiant demande à la formation suivante (indice = indice + 1)


> __Question 4 (Gale-Shapley)__ : Ecrire une fonction `galeShapley(etudiantPref,formationPref):` qui renvoie une proposition de mariage suivant l'agorithme de Gale-Shapley en fonction des listes de préférences des étudiants et des formations.

In [11]:
def galeShapley(etudiantPref,formationPref,complexcount=False):
    """
    renvoie un mariage stable
    
    
   :param array etudiantPref: la liste des préférences des étudiants
   :param array formationPref: la liste des préférences des formations
   :return: mariage stable
   :rtype: array
    """    
    return [0]

    
try:
    assert galeShapley(etudiantPref, formationPref) == [3, 1, 0, 4, 2] 
    print("galeShapley : OK")
except:
    print("galeShapley : ERREUR")


galeShapley : ERREUR


## Analyse (Séance 2)

Pour étudier les mariages obtenus, nous avons besoin d'utiliser l'algorithme sur plusieurs exemples. 

> __Question 5 (Préférences aléatoires)__ : 
Ecrire une fonction `listPrefGenerator(n)` qui renvoie une liste de <tt>n</tt> listes de préférences contenant des nombres aléatoires de 0 à n-1 (sans doublons dans une même liste). 

Par exemple : `listPrefGenerator(5)` pourrait renvoyer `[[0, 3, 2, 1, 4], [2, 3, 0, 4, 1], [2, 3, 1, 0, 4], [0, 3, 1, 2, 4], [0, 2, 3, 4, 1]]`

In [12]:
def listPrefGenerator(n):
    """
    renvoie une liste de préférences (liste de liste) générée aléatoirement avec des nombres de 0 à n-1
    
    
   :param n taille de la liste de préférence 
   :return: liste de préférences
   :rtype: array
    """  
    
    
    return [0]

n = 5
prefList = listPrefGenerator(n)
print(prefList)

[0]


> __Question 6 (compléxité moyenne)__ : Quelle est la complexité en pire cas ? Tracer la complexité en pratique sur des instances aléatoires de tailles différentes. De quelle complexité théorique se rapproche t'elle ? N'hésitez pas à rajouter un compteur d'opérations sur votre fonction `galeShapley`

Réponse : 

On appelle "regret" de l'individu $i$ la position du partenaire après mariage dans la liste des préférence de $i$.
Par exemple, si la liste de préférence de notre individu est <tt>[0, 3, 2, 1, 4]</tt> et qu'il est marié avec 3, son regret est de 1. Par contre, s'il est marié avec 4, son regret est de 4. 
Plus le regret de $i$ est faible, plus le mariage est réussi pour $i$. Evidemment, le regret du partenaire est également à prendre en compte pour connaitre la réussite (absolue) du mariage ! 

> __Question 7 (Regret)__ : Ecrire une fonction `regret(e, mariage, etudPref, formaPref, isStudent)` qui donne le regret de <tt>e</tt> étant donné le <tt>mariage</tt> et les listes de préférence. Attention, <tt>e</tt> est un étudiant si <tt>isStudent</tt> est à <tt>True</tt> et une formation sinon. 

In [13]:
def regret(e, mariage, etudPref, formaPref, isStudent):
    """
    renvoie le regret de e
    
    
   :param int e indice d'un etudiant ou d'une université
   :param array mariage résultat d'un mariage
   :param array etudPref liste des préférences des étudiants
   :param array formaPref liste des préférences des formations
   :param bool isStudent: permet de connaitre le rôle de e 
   
   :return: regret
   :rtype: int
    """  
    
    
    return -1

try:
    etudiantPref=[[1,0,2,4,3],[0,2,1,4,3],[2,0,4,1,3],[2,1,4,3,0],[0,1,2,3,4]]
    formationPref=[[2,0,4,3,1],[1,0,4,3,2],[4,1,2,3,0],[1,0,4,3,2],[2,1,4,3,0]]
    mariage = [3, 1, 0, 4, 2] #Mariage obtenu par galeShapley
    assert regret(0,mariage,etudiantPref, formationPref, False ) == 0
    assert regret(4,mariage,etudiantPref, formationPref, False ) == 3 #L'universite 4 est avec l'etudiant 3 donc regret de 3
    assert regret(0,mariage,etudiantPref, formationPref, True ) == 4
    assert regret(1,mariage,etudiantPref, formationPref, True ) == 2
    print("regret : OK")
except:
    print("regret : ERREUR")
    

regret : ERREUR


> __Question 8 (Regret, suite)__ : Ecrire une fonction `regrets(mariage, etudPref, formaPref)` qui renvoie une liste de deux listes : la première correspond au regrets des étudiants et la seconde aux regrets des formations. 

In [14]:
def regrets(mariage, etudPref, formaPref):
    """
    renvoie une liste des regrets des étudiants et des regrets des formations
    
   :param array mariage résultat d'un mariage
   :param array etudPref liste des préférences des étudiants
   :param array formaPref liste des préférences des formations
   
   :return: regrets
   :rtype: array
    """  
    
    return [0]

try:
    etudiantPref=[[1,0,2,4,3],[0,2,1,4,3],[2,0,4,1,3],[2,1,4,3,0],[0,1,2,3,4]]
    formationPref=[[2,0,4,3,1],[1,0,4,3,2],[4,1,2,3,0],[1,0,4,3,2],[2,1,4,3,0]]
    mariage = [3, 1, 0, 4, 2] #Mariage obtenu par galeShapley
    assert regrets(mariage, etudiantPref, formationPref) == [[4, 2, 1, 2, 2], [0, 0, 0, 1, 3]]
    print("regrets : OK")
except:
    print("regrets : ERREUR")

regrets : ERREUR


> __Question 9 (Indicateurs de regret)__ : Calculer les regrets sur plusieurs exemples, quels indicateurs vous semblent pertinents ? Comment évoluent les regrets avec la tailles des populations ? 

> __Question 10 (Conclusion)__ : Qu'en concluez vous ? 